In [ ]:
import json
import os
from ovejero import forward_modeling, model_trainer
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import matplotlib

def NOTIMPLEMENTED():
    raise NotImplementedError('Must specify config/save path')

# Forward Modeling Demo

__Author:__ Sebastian Wagner-Carena

__Last Run:__ 08/15/2020

__Goals:__ Learn how to compare the performance of the BNN model to a forward modeling approach

__Before running this notebook:__ Run the Train_Toy_Model notebook to understand how to train a model. Then train a model with whatever configuration you want. You will have to add the path to the config file in this notebook.

To start, all we have to do is load up our model weights and run it on a specific image in the validation set. Thankfully, that's pretty easy to do with the ForwardModeling class. If you don't have a GPU, generating 10000 samples for one image should still be managable but will take some time.

In [ ]:
# First specify the config path
config_path = NOTIMPLEMENTED()

# Check that the config has what you need
cfg = model_trainer.load_config(config_path)

# The ForwardModeling will do all the heavy lifting of preparing the model from the configuration file,
# initializing the validation dataset, and providing outputs correctly marginalized over the BNN uncertainties.
fow_model = forward_modeling.ForwardModel(cfg)

Now we need to select an image for the forward modeling comparison.

In [ ]:
image_index = 25
fow_model.select_image(image_index)

Now we can run our forward modeling. I would recommend saving the weights in case you want to stop and restart later. The 100 steps we do here aren't going to be enough for convergence, but for the demo this will be good enough.

In [ ]:
# Initialize our sampler for the three models
walker_ratio = 50
n_samps = 100
save_path_chains = os.path.join(root_path,'demos/demo_fow_model_chains.h5')

fow_model.initialize_sampler(walker_ratio,save_path_chains)
fow_model.run_sampler(n_samps)

You can plot the chains as a quick visual inspection that things are working. Feel free to some more advanced tests on the chains as well (for example take a look at the hierarchical inference notebook). Given that we've only done 100 steps, you shouldn't see anything that looks like convergence. The solid black line is the true parameter values. Feel free to put in larger numbers for good results.

In [ ]:
burnin = 0
fow_model.plot_chains(burnin=burnin)

We also want BNN samples of this one lens to make a comparison. Since we're only running the analysis on one lens we can afford to do some denser sampling (say 10k samples). Note, if you've only run 100 forward modeling steps and trained the BNN, the BNN will probably look a lot better than the forward model here. That's just a trick of convergence, run the forward model for longer.

In [ ]:
num_samples = 10000
sample_save_dir = os.path.join(root_path,'demos/fow_model_bnn_samps')
# Now let's look at the corner plot of the parameters we care about (the lens parameters)
color_map = ['#d95f02','#7570b3','#000000']
truth_color = '#e7298a'

# For an example of how to specify plot limits see the forward modeling plotting notebook in the paper folder. The
# standard we use here is the same as for the corner.corner package.
plot_limits = None
fontsize = 20

fig = fow_model.plot_posterior_contours(burnin,num_samples,sample_save_dir=sample_save_dir,
                                            color_map=color_map,plot_limits=plot_limits,truth_color=truth_color,
                                            save_fig_path=None,show_plot=False,add_legend=False,
                                            fontsize=fontsize)

handles = [Line2D([0], [0], color=color_map[0], lw=10),
           Line2D([0], [0], color=color_map[1], lw=10)]
fig.legend(handles,[r'Full BNN 0.1% Dropout','Forward Modeling'],loc=(0.525,0.73),
           fontsize=fontsize,framealpha=1.0)

Clean up the files we generated during the demo

In [ ]:
os.remove('demo_fow_model_chains.h5')